# Train GPT-2 on OpenWebText w/ Distributionally Robust Optimization

See python file of the same name as this notebook for the production version.

# Train Model

In [1]:
import huggingface_hub as hf_hub

def read_key(filename):
    with open(filename) as f:
        key = f.read().strip()
    return key

hf_hub.login(token=read_key('huggingface.key'), write_permission=True, add_to_git_credential=True)



Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/kdbanman/.cache/huggingface/token
Login successful


In [2]:
from tqdm.notebook import tqdm

In [3]:
from datasets import load_from_disk
from torch.utils.data.dataloader import DataLoader

batch_size = 32

tokenized_datasets = load_from_disk('tokenized-openwebtext')

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=batch_size)

In [4]:
from torch.nn import CrossEntropyLoss
import torch


def dro_loss(inputs, logits, alpha=0.8):
    # Shift so that tokens < n predict n
    shift_labels = inputs[..., 1:].contiguous()
    shift_logits = logits[..., :-1, :].contiguous()
    
    # Calculate per-token loss
    loss_fct = CrossEntropyLoss(reduce=False)
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    
    # Resize and average loss per sample
    loss_per_sample = loss.view(shift_logits.size(0), shift_logits.size(1)).mean(axis=1)

    if alpha < 1.0:
        # Keep only largest alpha-fraction of losses by reweighting smallest (1-alpha)-fraction to zero
        num_samples = len(loss_per_sample)
        num_to_ignore = num_samples - int(num_samples * alpha)

        if num_to_ignore >= 1 or num_to_ignore < num_samples:
            cutoff_value, _cutoff_index = torch.kthvalue(loss_per_sample, num_to_ignore, dim=0)
            loss_per_sample[loss_per_sample < cutoff_value] = 0
        else:
            print("ERROR: crazy reweighting request from the following.  Skipping DRO reweighting.")
            priunt(f'alpha: {alpha}')
            priunt(f'num_samples: {num_samples}')
            priunt(f'num_to_ignore: {num_to_ignore}')
            print(f'losses: {loss_per_sample}')
    
    return loss_per_sample.mean()

In [5]:
weight_decay = 0.1


def get_grouped_params(model, no_decay=["bias", "LayerNorm.weight"]):
    params_with_wd, params_without_wd = [], []
    for n, p in model.named_parameters():
        if any(nd in n for nd in no_decay):
            params_without_wd.append(p)
        else:
            params_with_wd.append(p)
    return [
        {"params": params_with_wd, "weight_decay": weight_decay},
        {"params": params_without_wd, "weight_decay": 0.0},
    ]

In [6]:
def evaluate(max_eval_batches=None):
    model.eval()
    losses = []

    if max_eval_batches is None:
        max_eval_batches = len(eval_dataloader)
        
    for step, batch in tqdm(
        enumerate(eval_dataloader), total=max_eval_batches
    ):
        with torch.no_grad():
            outputs = model(batch["input_ids"], labels=batch["input_ids"])

        losses.append(accelerator.gather(outputs.loss))
        
        if step >= max_eval_batches:
            break

    if len(losses[0].shape) == 0:
        loss = torch.mean(torch.stack(losses))
    else:
        loss = torch.mean(torch.cat(losses))
        
    try:
        perplexity = torch.exp(loss)
    except OverflowError:
        perplexity = float("inf")
    return loss.item(), perplexity.item()

In [7]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig

# For now, the dataset is tokenized in advance using context_length, so this value is fixed
# at tokenization time.  In the future, tokenization should really be streamed.  Then 
# context_length can be varied.
context_length = 256

tokenizer = AutoTokenizer.from_pretrained("gpt2")
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

In [8]:
model = GPT2LMHeadModel(config)

In [9]:
from torch.optim import AdamW

optimizer = AdamW(get_grouped_params(model), lr=5e-4)

In [10]:
from accelerate import Accelerator

accelerator = Accelerator(cpu=False)

model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [11]:
from transformers import get_scheduler

num_train_epochs = 1
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=1_000,
    num_training_steps=num_training_steps,
)

In [12]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "gpt2-openwebtext-dro-test"
repo_name = get_full_repo_name(model_name)
repo_name

'kdbanman/gpt2-openwebtext-dro-test'

In [13]:
import huggingface_hub

try:
    huggingface_hub.model_info(repo_name)
except huggingface_hub.utils.RepositoryNotFoundError:
    huggingface_hub.create_repo(repo_name)

In [14]:
output_dir = model_name
repo = Repository(output_dir, clone_from=repo_name)

/home/kdbanman/tmp/gpt2-openwebtext-dro-test is already a clone of https://huggingface.co/kdbanman/gpt2-openwebtext-dro-test. Make sure you pull the latest changes with `repo.git_pull()`.


In [15]:
dro_alpha = 0.8

epoch_step_logging_interval = 8

gradient_accumulation_steps = 8
gradient_step_eval_interval = 30
gradient_steps_since_eval = 0
gradient_steps = 0

num_eval_batches = 50

model.train()
for epoch in range(1, num_train_epochs + 1):
    for epoch_step, batch in tqdm(
        enumerate(train_dataloader, start=1), total=len(train_dataloader)
    ):
        logits = model(batch["input_ids"]).logits
        loss = dro_loss(batch["input_ids"], logits, alpha=dro_alpha)
        if epoch_step % epoch_step_logging_interval == 0:
            accelerator.print(
                {
                    "lr": lr_scheduler.get_lr(),
                    "samples/contexts": epoch * epoch_step * batch_size,
                    "samples/tokens": epoch * epoch_step * batch_size * context_length,
                    "gradient_steps": gradient_steps,
                    "loss/train": loss.item() * gradient_accumulation_steps,
                }
            )
        loss = loss / gradient_accumulation_steps
        accelerator.backward(loss)
        if epoch_step % gradient_accumulation_steps == 0:
            accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            gradient_steps += 1
            gradient_steps_since_eval += 1
            
        if gradient_steps_since_eval >= gradient_step_eval_interval:
            gradient_steps_since_eval = 0
            eval_loss, perplexity = evaluate(num_eval_batches)
            accelerator.print({"loss/eval": eval_loss, "perplexity": perplexity})
            model.train()
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
            if accelerator.is_main_process:
                tokenizer.save_pretrained(output_dir)
                repo.push_to_hub(
                    commit_message=f"Training in progress epoch step {epoch * epoch_step}", blocking=False
                )

accelerator.end_training()

  0%|          | 0/878614 [00:00<?, ?it/s]

/home/kdbanman/tmp/env/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


{'lr': [0.0, 0.0], 'samples/contexts': 256, 'samples/tokens': 65536, 'gradient_steps': 0, 'loss/train': 71.26820373535156}
{'lr': [5e-07, 5e-07], 'samples/contexts': 512, 'samples/tokens': 131072, 'gradient_steps': 1, 'loss/train': 71.38358306884766}
{'lr': [1e-06, 1e-06], 'samples/contexts': 768, 'samples/tokens': 196608, 'gradient_steps': 2, 'loss/train': 71.17308044433594}
{'lr': [1.5e-06, 1.5e-06], 'samples/contexts': 1024, 'samples/tokens': 262144, 'gradient_steps': 3, 'loss/train': 70.84748840332031}
{'lr': [2e-06, 2e-06], 'samples/contexts': 1280, 'samples/tokens': 327680, 'gradient_steps': 4, 'loss/train': 70.48038482666016}
{'lr': [2.5e-06, 2.5e-06], 'samples/contexts': 1536, 'samples/tokens': 393216, 'gradient_steps': 5, 'loss/train': 69.88656616210938}
{'lr': [3e-06, 3e-06], 'samples/contexts': 1792, 'samples/tokens': 458752, 'gradient_steps': 6, 'loss/train': 69.14768981933594}
{'lr': [3.5e-06, 3.5e-06], 'samples/contexts': 2048, 'samples/tokens': 524288, 'gradient_steps': 

  0%|          | 0/50 [00:00<?, ?it/s]

{'loss/eval': 9.393847465515137, 'perplexity': 12014.2353515625}


...

  0%|          | 0/50 [00:00<?, ?it/s]

{'loss/eval': 5.631961345672607, 'perplexity': 279.209228515625}


/home/kdbanman/tmp/env/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


{'lr': [0.000255, 0.000255], 'samples/contexts': 130816, 'samples/tokens': 33488896, 'gradient_steps': 510, 'loss/train': 36.41244125366211}
{'lr': [0.00025550000000000003, 0.00025550000000000003], 'samples/contexts': 131072, 'samples/tokens': 33554432, 'gradient_steps': 511, 'loss/train': 36.874427795410156}
{'lr': [0.000256, 0.000256], 'samples/contexts': 131328, 'samples/tokens': 33619968, 'gradient_steps': 512, 'loss/train': 36.71207046508789}
{'lr': [0.0002565, 0.0002565], 'samples/contexts': 131584, 'samples/tokens': 33685504, 'gradient_steps': 513, 'loss/train': 38.00460433959961}
{'lr': [0.000257, 0.000257], 'samples/contexts': 131840, 'samples/tokens': 33751040, 'gradient_steps': 514, 'loss/train': 37.89382553100586}
{'lr': [0.0002575, 0.0002575], 'samples/contexts': 132096, 'samples/tokens': 33816576, 'gradient_steps': 515, 'loss/train': 37.34954833984375}
{'lr': [0.00025800000000000004, 0.00025800000000000004], 'samples/contexts': 132352, 'samples/tokens': 33882112, 'gradien

KeyboardInterrupt: 